### Section is to identify whether the image is a horse or a human. 
- images are colored. Extract real images and need to be converted to 3D matrices

In [13]:
import zipfile
import sys
import warnings
import os
import h5py
import numpy as np
import joblib as j
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D,Flatten, Dropout
from tensorflow.keras.optimizers.legacy import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
warnings.filterwarnings('ignore')

#### Extracting the training and validation set into their respective directories

In [2]:
train_file_name, validation_file_name = "horse-or-human.zip", "validation-horse-or-human.zip"
training_dir,validation_dir = 'horse-or-human/training/','horse-or-human/validation/'
RETRAIN_model1 = True ## model1 is a custom nueral network built from scratch. If retrain = False the model will be loaded from memory (custom_best_model.h5)
RETRAIN_model2 = True ## model2 is a an image classifier that is an extension of InceptionV3 model. If retrain = True the model will be trained on the dataset else it will load the best parameters from memory (pretrained_best_model.h5)
with zipfile.ZipFile(train_file_name, 'r') as train_zipref:
    train_zipref.extractall(training_dir)

with zipfile.ZipFile(validation_file_name, 'r') as validation_zipref:
    validation_zipref.extractall(validation_dir)

#### We use image Augmentation techniques for improved Accuracy
- We also define callbacks on models for stopping training the model with poor accuracy metrics

In [3]:
early_stopping_callback = EarlyStopping(patience=5,monitor='val_loss',verbose=1,mode='min')
model_checkpoint_callback_custom = ModelCheckpoint('custom_best_model.h5', monitor='val_loss', save_best_only=True,verbose=1, save_freq='epoch')
model_checkpoint_callback_pretrained = ModelCheckpoint('pretrained_best_model.h5', monitor='val_loss', save_best_only=True,verbose=1, save_freq='epoch')

In [4]:
train_data_gen,validation_data_gen = ImageDataGenerator(rescale=1./255,rotation_range=35,width_shift_range=0.1,height_shift_range=.25,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest'),ImageDataGenerator(rescale=1./255,rotation_range=35,width_shift_range=0.1,height_shift_range=.25,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
train_generator,validation_generator = train_data_gen.flow_from_directory(training_dir,target_size=(300,300),class_mode='binary'),validation_data_gen.flow_from_directory(validation_dir,target_size=(300,300),class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [5]:
model = Sequential([
  Conv2D(16,(3,3),input_shape=(300,300,3),activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(32,(3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(64,(3,3), activation='relu'),
  MaxPooling2D(2,2),
  Flatten(),
  Dense(512,activation='relu'),
  Dropout(0.2),
  Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001),metrics=['accuracy'])

In [6]:
if((RETRAIN_model1) or (not os.path.exists('custom_best_model.h5'))):
  history = model.fit(train_generator,epochs=15,validation_data=validation_generator,callbacks=[early_stopping_callback,model_checkpoint_callback_custom]) 
else :
  model = load_model('custom_best_model.h5')
  print("Best model loaded from memory.")  

Best model loaded from memory.


#### Testing some real images with the trained model
- Even though the validation accuracy is moderate the classification is very 100% accurate on the test_images

In [7]:
file_name = "./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
if not os.path.isfile(file_name):
    print("File not found:", file_name)
    sys.exit("Execution stopped because the file was not found")
else:
    print("File exists.")
    
try:
    with h5py.File(file_name, 'r') as f:
        print("File is a valid HDF5 file.")
except OSError as e:
    print("File is not a valid HDF5 file.",e,)
    sys.exit("File was corrupted or of an invalid format")        

File exists.
File is a valid HDF5 file.


In [8]:
file_name = "./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
pretrained_model = InceptionV3(input_shape=(300,300,3),include_top=False,weights=None)
pretrained_model.load_weights(file_name)
pretrained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d_5[0][0]']               
 alization)                     )                                                      

In [9]:
for layer in pretrained_model.layers:
  layer.trainable = False
stop_layer = pretrained_model.get_layer('mixed7')
x = Flatten()(stop_layer.output)
x = Dense(1024,activation='relu')(x)
x = Dense(1,activation='sigmoid')(x)

In [10]:
model2 = None
if (RETRAIN_model2 or (not os.path.exists('pretrained_best_model.h5'))):
  model2 = Model(pretrained_model.input,x)
  model2.compile(optimizer=RMSprop(learning_rate=0.0001),loss='binary_crossentropy',metrics=['acc'])
  model2.fit(train_generator,epochs=40,validation_data=validation_generator,callbacks=[early_stopping_callback,model_checkpoint_callback_pretrained])
else: 
  model2 = load_model('pretrained_best_model.h5')  
  print("pretrained_model loaded from memory!")

pretrained_model loaded from memory!


#### Testing images

In [21]:
test_dir_horse = './test_images/horse'
test_dir_human = './test_images/human'
print(train_generator.class_indices)
horse_image_files = [f for f in os.listdir(test_dir_horse) if f.endswith(('jpg', 'png','jpeg'))]
human_image_files = [f for f in os.listdir(test_dir_human) if f.endswith(('jpg', 'png','jpeg'))]
correct_pred_horses_model_1 = 0
correct_pred_horses_model_2 = 0
correct_pred_humans_model_1 = 0
correct_pred_humans_model_2 = 0

for img_file in horse_image_files:
    img_path = os.path.join(test_dir_horse, img_file)
    img = image.load_img(img_path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    classes2 = model2.predict(image_tensor)
    if classes[0] > 0.5:
        print(f"{model.name} - {img_file} is a human")
    else:
        print(f"{model.name} - {img_file} is a horse")
        correct_pred_horses_model_1 += 1
    
    if classes2[0] > 0.5:
        print(f"{model2.name} - {img_file} is a human")
    else:
        print(f"{model2.name} - {img_file} is a horse")
        correct_pred_horses_model_2 += 1
print(f"Accuracy of horses (custom CNN) = {float(correct_pred_humans_model_1)/len(horse_image_files)}")        
print(f"Accuracy of horses (pretrained model Inception V3) = {float(correct_pred_humans_model_2)/len(horse_image_files)}")        
 
for img_file in human_image_files:
    img_path = os.path.join(test_dir_human, img_file)
    img = image.load_img(img_path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    classes2 = model2.predict(image_tensor)
    if classes[0] > 0.5:
        print(f"{model.name} - {img_file} is a human")
        correct_pred_humans_model_1 += 1
    else:
        print(f"{model.name} - {img_file} is a horse")
    if classes2[0] > 0.5:
        print(f"{model2.name} - {img_file} is a human")
        correct_pred_humans_model_2 += 1  
    else:
        print(f"{model2.name} - {img_file} is a horse")   
print(f"Accuracy of humans (custom CNN) = {float(correct_pred_humans_model_1)/len(human_image_files)}")        
print(f"Accuracy of humans (pretrained model Inception V3) = {float(correct_pred_humans_model_2)/len(human_image_files)}")           

{'horses': 0, 'humans': 1}
1/1 [==============================] - 0s 131ms/step
sequential_2 - image-31.png is a horse
model_3 - image-31.png is a human
1/1 [==============================] - 0s 100ms/step
sequential_2 - image-25.png is a horse
model_3 - image-25.png is a human
1/1 [==============================] - 0s 98ms/step
sequential_2 - image-24.png is a horse
model_3 - image-24.png is a human
1/1 [==============================] - 0s 102ms/step
sequential_2 - image-30.png is a human
model_3 - image-30.png is a human
1/1 [==============================] - 0s 101ms/step
sequential_2 - image-26.png is a human
model_3 - image-26.png is a human
1/1 [==============================] - 0s 106ms/step
sequential_2 - image-32.png is a horse
model_3 - image-32.png is a human
1/1 [==============================] - 0s 99ms/step
sequential_2 - image-33.png is a horse
model_3 - image-33.png is a human
1/1 [==============================] - 0s 112ms/step
sequential_2 - image-27.png is a horse
m